In [ ]:
# Importando os pacotes 
import numpy as np
import pandas as pd
import scipy.stats
import sklearn
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.metrics import homogeneity_completeness_v_measure
from sklearn.preprocessing import MinMaxScaler

In [ ]:
# Carregando os dados
df = pd.read_csv('dataset.csv', sep = ',', encoding = 'utf-8')

In [ ]:
# Renomeando as colunas
colunas = ['Id', 'Sexo', 'Estado_Civil', 'Idade', 'Educacao', 'Renda', 'Ocupacao', 'Tamanho_Cidade']
df.columns = colunas

In [ ]:
# Convertendo as variáveis Id, Sexo, Estado_Civil, Educacao, Ocupacacao e Tamanho_Cidade para categorias
df['Id'] = pd.Categorical(df['Id'])
df['Sexo'] = pd.Categorical(df['Sexo'])
df['Estado_Civil'] = pd.Categorical(df['Estado_Civil'])
df['Educacao'] = pd.Categorical(df['Educacao'])
df['Ocupacao'] = pd.Categorical(df['Ocupacao'])
df['Tamanho_Cidade'] = pd.Categorical(df['Tamanho_Cidade'])

In [ ]:
# Criando uma cópia do dataframne
df1 = df.copy()

In [ ]:
# Colunas numéricas (quantitativas) e categóricas 
colunas_numericas = ['Idade', 'Renda']
colunas_categoricas = ['Id', 'Sexo', 'Estado_Civil', 'Educacao', 'Ocupacao', 'Tamanho_Cidade']

In [ ]:
# Dataframes com os tipos diferentes de variáveis
df_num = df1[colunas_numericas]
df_cat = df1[colunas_categoricas]

In [ ]:
# Sumário estatístico das variáveis numéricas
sumario_num = pd.DataFrame(index = df_num.columns)
sumario_num['Não Nulos'] = df_num.count().values
sumario_num['Valores Únicos'] = df_num.nunique().values
sumario_num['Média'] = df_num.mean()
sumario_num['DesvPad'] = df_num.std()
sumario_num['Min'] = df_num.min()
sumario_num['Max'] = df_num.max()

sumario_num= sumario_num.astype(int)

In [ ]:
# Sumário estatístico das variáveis categóricas
sumario_cat = pd.DataFrame(index = df_cat.columns)
sumario_cat['Não Nulos'] = df_cat.count().values
sumario_cat['% Populado'] = round(sumario_cat['Não Nulos'] / df_cat.shape[0]*100,2)
sumario_cat['Valores Únicos'] = df_cat.nunique().values

# Adiciona mais uma coluna com valores mais comuns
temp = []
for coluna in colunas_categoricas:
    temp.append(df_cat[coluna].value_counts().idxmax())
sumario_cat['Valores Mais Comuns'] = temp

sumario_cat= sumario_cat.astype(int)

In [ ]:
# Aplicando redução de dimensionalidade
pca = PCA(n_components = 2).fit_transform(df1)

In [ ]:
# Determinando um range de K
k_range = range(1,10)

In [ ]:
# Aplicando o modelo K-Means para cada valor de K (esta célula pode levar bastante tempo para ser executada)
k_means_var = [KMeans(n_clusters = k).fit(pca) for k in k_range]

In [ ]:
# Ajustando o centróide do cluster para cada modelo
centroids = [X.cluster_centers_ for X in k_means_var]

In [ ]:
# Calculando a distância euclidiana de cada ponto de dado para o centróide
from scipy.spatial.distance import cdist, pdist
k_euclid = [cdist(pca, cent, 'euclidean') for cent in centroids]
dist = [np.min(ke, axis = 1) for ke in k_euclid]

In [ ]:
# Soma dos quadrados das distâncias dentro do cluster
soma_quadrados_intra_cluster = [sum(d**2) for d in dist]

In [ ]:
# Soma total dos quadrados
soma_total = sum(pdist(pca)**2)/pca.shape[0]

In [ ]:
# Soma dos quadrados entre clusters
soma_quadrados_inter_cluster = soma_total - soma_quadrados_intra_cluster

In [ ]:
# Criação do modelo
modelo_v1 = KMeans(n_clusters = 4, 
                     init = 'k-means++', 
                     n_init = 10, 
                     max_iter = 300, 
                     tol = 0.0001,   
                     algorithm = 'elkan')

In [ ]:
# Treinamento do modelo
modelo_v1.fit(pca)

In [ ]:
# Extração dos labels
labels1 = modelo_v1.labels_

In [ ]:
previsoes = modelo_v1.fit_predict(pca)

In [ ]:
# Converte o array para dataframe
df_labels1 = pd.DataFrame(labels1)

In [ ]:
# Vamos fazer o merge de pca e os labels (clusters) encontrados pelo modelo
df_final1 = df1.merge(df_labels1, left_index = True, right_index = True)
df_final1.rename(columns = {0:"Cluster"}, inplace = True)

In [ ]:
cluster1 = df_final1['Cluster'].value_counts().rename_axis('Cluster').reset_index(name = 'Total')
cluster1

In [ ]:
df_final2 = df_final1.copy()
cols3 = df_final2.columns.tolist()
cols3 = cols3[-1:] + cols3[:-1]
df_final2 = df_final2[cols3]
df_final2.head(5)

In [ ]:
resumo_cluster = df_final1.groupby('Cluster')['Renda'].agg([('Clientes', 'count'),('Media_Renda', 'mean'), ('DesvPad', 'std'), 
('Mínimo', 'min'), 
('Máximo', 'max')]) 
resumo_cluster= resumo_cluster.astype(int)
resumo_cluster['Cluster']= [0, 1, 2, 3]
cols4 = resumo_cluster.columns.tolist()
cols4 = cols4[-1:] + cols4[:-1]
resumo_cluster = resumo_cluster[cols4]

In [ ]:
resumo_cluster

In [ ]:
# Calcula a quantidade de clientes de cada genero por cluster
sexo_cluster = df_final2.groupby('Cluster')['Sexo'].value_counts(sort=False)
sexo_cluster

In [ ]:
contagem_sexo= sexo_cluster.values
contagem_sexo

In [ ]:
colunas_sexo = ['Cluster', 'Masculino', 'Feminino']
cluster_sexo = [0, 1, 2, 3]
df_sexo = pd.DataFrame(columns= colunas_sexo)
df_sexo['Cluster'] = cluster_sexo

In [ ]:
df_sexo.iloc[[0],1] = contagem_sexo[0]
df_sexo.iloc[[0],2] = contagem_sexo[1]
df_sexo.iloc[[1],1] = contagem_sexo[2]
df_sexo.iloc[[1],2] = contagem_sexo[3]
df_sexo.iloc[[2],1] = contagem_sexo[4]
df_sexo.iloc[[2],2] = contagem_sexo[5]
df_sexo.iloc[[3],1] = contagem_sexo[6]
df_sexo.iloc[[3],2] = contagem_sexo[7]
df_sexo

In [ ]:
# Calcula a quantidade de clientes de cada estado civil por cluster
civil_cluster = df_final2.groupby('Cluster')['Estado_Civil'].value_counts(sort=False)
civil_cluster

In [ ]:
contagem_civil= civil_cluster.values
contagem_civil

In [ ]:
colunas_civil = ['Cluster', 'Solteiro', 'Nao_Solteiro']
cluster_civil = [0, 1, 2, 3]
df_civil = pd.DataFrame(columns= colunas_civil)
df_civil['Cluster'] = cluster_civil

In [ ]:
df_civil.iloc[[0],1] = contagem_civil[0]
df_civil.iloc[[0],2] = contagem_civil[1]
df_civil.iloc[[1],1] = contagem_civil[2]
df_civil.iloc[[1],2] = contagem_civil[3]
df_civil.iloc[[2],1] = contagem_civil[4]
df_civil.iloc[[2],2] = contagem_civil[5]
df_civil.iloc[[3],1] = contagem_civil[6]
df_civil.iloc[[3],2] = contagem_civil[7]
df_civil

In [ ]:
# Calcula a quantidade de clientes de cada nível de formação por cluster
formacao_cluster = df_final2.groupby('Cluster')['Educacao'].value_counts(sort=False)
formacao_cluster

In [ ]:
contagem_formacao= formacao_cluster.values
contagem_formacao

In [ ]:
colunas_formacao = ['Cluster', 'Outro', 'Ensino_Medio', 'Universitario', 'Graduado']
cluster_formacao = [0, 1, 2, 3]
df_formacao = pd.DataFrame(columns= colunas_formacao)
df_formacao['Cluster'] = cluster_formacao

In [ ]:
df_formacao.iloc[[0],1] = contagem_formacao[0]
df_formacao.iloc[[0],2] = contagem_formacao[1]
df_formacao.iloc[[0],3] = contagem_formacao[2]
df_formacao.iloc[[0],4] = contagem_formacao[3]
df_formacao.iloc[[1],1] = contagem_formacao[4]
df_formacao.iloc[[1],2] = contagem_formacao[5]
df_formacao.iloc[[1],3] = contagem_formacao[6]
df_formacao.iloc[[1],4] = contagem_formacao[7]
df_formacao.iloc[[2],1] = contagem_formacao[8]
df_formacao.iloc[[2],2] = contagem_formacao[9]
df_formacao.iloc[[2],3] = contagem_formacao[10]
df_formacao.iloc[[2],4] = contagem_formacao[11]
df_formacao.iloc[[3],1] = contagem_formacao[12]
df_formacao.iloc[[3],2] = contagem_formacao[13]
df_formacao.iloc[[3],3] = contagem_formacao[14]
df_formacao.iloc[[3],4] = contagem_formacao[15]

df_formacao

In [ ]:
# Calcula a quantidade de clientes de cada ocupação por cluster
ocupacao_cluster = df_final2.groupby('Cluster')['Ocupacao'].value_counts(sort=False)
ocupacao_cluster

In [ ]:
contagem_ocupacao= ocupacao_cluster.values
contagem_ocupacao

In [ ]:
colunas_ocupacao = ['Cluster', 'Desempregado', 'Funcionario', 'Gestao/Autonomo']
cluster_ocupacao = [0, 1, 2, 3]
df_ocupacao = pd.DataFrame(columns= colunas_ocupacao)
df_ocupacao['Cluster'] = cluster_ocupacao

In [ ]:
df_ocupacao.iloc[[0],1] = contagem_ocupacao[0]
df_ocupacao.iloc[[0],2] = contagem_ocupacao[1]
df_ocupacao.iloc[[0],3] = contagem_ocupacao[2]
df_ocupacao.iloc[[1],1] = contagem_ocupacao[3]
df_ocupacao.iloc[[1],2] = contagem_ocupacao[4]
df_ocupacao.iloc[[1],3] = contagem_ocupacao[5]
df_ocupacao.iloc[[2],1] = contagem_ocupacao[6]
df_ocupacao.iloc[[2],2] = contagem_ocupacao[7]
df_ocupacao.iloc[[2],3] = contagem_ocupacao[8]
df_ocupacao.iloc[[3],1] = contagem_ocupacao[9]
df_ocupacao.iloc[[3],2] = contagem_ocupacao[10]
df_ocupacao.iloc[[3],3] = 0

df_ocupacao

In [ ]:
# Calcula a quantidade de clientes de cada tamanho de cidade por cluster
cidade_cluster = df_final2.groupby('Cluster')['Tamanho_Cidade'].value_counts(sort=False)
cidade_cluster

In [ ]:
contagem_cidade= cidade_cluster.values
contagem_cidade

In [ ]:
colunas_cidade = ['Cluster', 'Pequena', 'Media', 'Grande']
cluster_cidade = [0, 1, 2, 3]
df_cidade = pd.DataFrame(columns= colunas_cidade)
df_cidade['Cluster'] = cluster_cidade

In [ ]:
df_cidade.iloc[[0],1] = contagem_cidade[0]
df_cidade.iloc[[0],2] = contagem_cidade[1]
df_cidade.iloc[[0],3] = contagem_cidade[2]
df_cidade.iloc[[1],1] = contagem_cidade[3]
df_cidade.iloc[[1],2] = contagem_cidade[4]
df_cidade.iloc[[1],3] = contagem_cidade[5]
df_cidade.iloc[[2],1] = contagem_cidade[6]
df_cidade.iloc[[2],2] = contagem_cidade[7]
df_cidade.iloc[[2],3] = contagem_cidade[8]
df_cidade.iloc[[3],1] = contagem_cidade[9]
df_cidade.iloc[[3],2] = contagem_cidade[10]
df_cidade.iloc[[3],3] = contagem_cidade[11]

df_cidade

In [ ]:
cluster0= df_final2[df_final2.Cluster.eq(0)]
cluster0